The goal of this script is to identify DEG based on FC and P_Value , it is dicussed to try with P-value and adjusted pvalue

In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.multitest import multipletests


In [ ]:
def calculate_adj_pvalues(data):
    column_pvalues_stored = []
    Filtered_data = data.copy()

    for column in data.columns:
        parts = column.split()
        if len(parts) > 3 and parts[3] == 'hr(p)':
            column_pvalues_stored.append(column)
    

    if column_pvalues_stored:
        qvalues = data[column_pvalues_stored].apply(lambda col: multipletests(col, method='fdr_bh')[1])
        Filtered_data[column_pvalues_stored] = qvalues

    return Filtered_data

In [10]:
def selection_by_logFC(full_data,log_value):
    '''The aim of this function is to extract upregulated and downregulated genes'''
    log_columns=[]
    for column in full_data.columns[2:]:
        parts= column.split()
        if parts[3] != 'hr(p)':
            log_columns.append(column)
            #extract genes based on upregaulted or downregulated
            filtered_by_logFC = full_data.loc[
                (full_data[log_columns].abs() > log_value).any(axis=1)
            ]
            
    return filtered_by_logFC
    

In [11]:
def selection_by_Pvalue(log_data,P_value):
    '''The aim of this function is to extract significant genes'''
    pvalue_columns=[]
    for column in log_data.columns[2:]:
        parts = column.split()
        #P-value column 
        if parts[3] == 'hr(p)':
            pvalue_columns.append(column)
            #extract genes based on upregaulted or downregulated
            filtered_by_Pvalue = log_data.loc[
                (log_data[pvalue_columns] < P_value).any(axis=1)
            ]
            
    return filtered_by_Pvalue
    

In [12]:

def selection_by_Adjusted_P_value(log_data_adj, pvalue_adj=0.05):
    '''The aim of this function is to extract significant genes based on adjusted p-values.'''
    pvalue_adjusted_columns = []

    # Identify p-value columns
    for column in log_data_adj.columns[2:]:
        parts = column.split()
        if parts[3] == 'hr(p)':  # Ensure there are enough parts
            pvalue_adjusted_columns.append(column)
    
    # Filter rows based on the p-value threshold
    if pvalue_adjusted_columns:
        filtered_by_Pvalue_adj= log_data_adj.loc[
            (log_data_adj[pvalue_adjusted_columns] < pvalue_adj).any(axis=1)
        ]
    else:
        filtered_by_Pvalue_adj= pd.DataFrame()  # Return an empty DataFrame if no p-value columns are found
    
    return filtered_by_Pvalue_adj



    